In [1]:
# Installing the libraries for project
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install spacy-transformers
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 59.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/8f/bc/213638f0242858d936c171f4e32e431a6fbbe703184a944a81c065cdc414/spacy-3.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 71.7 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.4
    Uninstalling spacy-3.5.4:
      Successfully uninstalled spacy-3.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.6.0 which is incompatible.
  Obtaining dependency information for spacy-transformers from https://files.pythonhosted.org/packages/16/40/892e39e29ed984b29507d69c4076429a3e9404b56e79c00e2a8d9b9cc1e8/spacy_transformers-1.2.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
 

In [2]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [3]:
def make_v3_dataset(data, db = []):
    nlp = spacy.blank('en')
    failed_record = []
    if not db:
        db = DocBin()
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot['entities']:
            span = doc.char_span(start, end, label = label, alignment_mode = 'contract')
            if span is None:
                print(f'empty entity, {text}, {annot["entities"]}') #I expect this to never happen
            else:
                ents.append(span)
        try:
            doc.ents = ents
        except:
            failed_record.append((text, annot))
        db.add(doc)
    return db, failed_record

In [34]:
tagged_data = '/content/drive/MyDrive/Dissertation/tagged_data'
list_tagged_files = os.listdir(tagged_data)

In [35]:
output_tagged_spacy_form ='/content/drive/MyDrive/Dissertation/output_tagged_spacy_form'

In [36]:
# Converting json annotated files to spacy format
for i in tqdm(range(len(list_tagged_files))):
  file_path = os.path.join(tagged_data,list_tagged_files[i])
  # print(file_path)
  with open(file_path,'r') as f:
    data = json.load(f)
  saving_path = os.path.join(output_tagged_spacy_form,list_tagged_files[i][:-5]+'.spacy')
  a,b = make_v3_dataset(data['annotations'])
  a.to_disk(saving_path)

 33%|███▎      | 33/100 [00:06<00:13,  5.10it/s]

empty entity, CONSULTANT
Profile
Skilled professional with 15 years of experience in electrical and quality engineering for numerous engineering 
and quality control processes.
Expertise in creating detailed test plans and test cases for optimal quality control. 
Solid background in various testing methodologies for both client-
facing and non-client-facing procedures 
and products.
Experience
Customer Relations
Â Monitored customer technical related issues
Â Maximized operational efficiency by coaching staff on various customer service initiatives
Â Improved employee-customer relations by providing social skills and managerial training to staff.
Analytical Skills
Oversaw quality assurance initiatives to design and implement products and processes for ongoing projects.
Ensured optimal productivity and quality assurance initiatives by performing routine maintenance on electrical equipment.
Review statistics and determine areas requiring enhancement, which includes inspecting samples and


100%|██████████| 100/100 [00:19<00:00,  5.15it/s]


In [43]:
import os
import shutil

# Define the source directory where your 100 samples are located
source_directory = "/content/drive/MyDrive/Dissertation/output_tagged_spacy_form"  # Adjust the path accordingly

# Define the destination directories for training and validation sets
training_directory = "/content/drive/MyDrive/Dissertation/training_data"  # Adjust the path accordingly
validation_directory = "/content/drive/MyDrive/Dissertation/validation_data"  # Adjust the path accordingly

# Create the destination directories if they don't exist
os.makedirs(training_directory, exist_ok=True)
os.makedirs(validation_directory, exist_ok=True)

# Get a list of files in the source directory
files = os.listdir(source_directory)

# Sort the files to ensure consistent splitting
files.sort()

# Separate files into training and validation sets
for idx, filename in enumerate(files, 1):
    # If the file index is a multiple of 5, move it to the validation set
    if idx % 5 == 0:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(validation_directory, filename)
    # Otherwise, move it to the training set
    else:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(training_directory, filename)

    # Move the file to the appropriate directory
    shutil.move(source_path, destination_path)


In [10]:
# Initialization setting
!python -m spacy init fill-config /content/drive/MyDrive/Dissertation/base_config.cfg /content/drive/MyDrive/Dissertation/config.cfg


2023-07-19 11:48:12.367142: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Dissertation/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [44]:
!python -m spacy train -g 0 /content/drive/MyDrive/Dissertation/config.cfg  --output /content/drive/MyDrive/Dissertation/generated_model/ --paths.train /content/drive/MyDrive/Dissertation/training_data --paths.dev /content/drive/MyDrive/Dissertation/validation_data

2023-07-19 16:01:03.833140: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory:
/content/drive/MyDrive/Dissertation/generated_model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-07-19 16:01:07,942] [INFO] Set up nlp object from config
[2023-07-19 16:01:07,959] [INFO] Pipeline: ['transformer', 'ner']
[2023-07-19 16:01:07,963] [INFO] Created vocabulary
[2023-07-19 16:01:07,963] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- Th